In [1]:
from metaxime.parser import ParserRP2
from biopathopt import ModelBuilder, Data
import numpy as np

In [2]:
from metaxime.cache_data import RR_Data

In [5]:
data = RR_Data()

In [16]:
RR_Data._rr

In [7]:
from metaxime.parser import ParserRP2
parser = ParserRP2(
    rp2_scope_path='bug/out_scope.csv',
    rp2_cmp_path='bug/out_compounds.csv',
    rp2_paths_path='bug/out_paths.csv',
)
all_models = parser.return_rp2_models(compartment_id='c')

ArgumentError: Python argument types in
    rdkit.Chem.rdinchi.InchiToMol(float, bool, bool)
did not match C++ signature:
    InchiToMol(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> inchi, bool sanitize=True, bool removeHs=True)

In [ ]:
parser.completed_paths[1]

In [74]:
type(parser.rr_recipes)

method

In [3]:
parser.rr_recipes['MNXR121985']

NameError: name 'parser' is not defined

{1: {'RR-02-55aae13ba05649cb-16-F': {'MNXR121985': {'MNXM2781': {'rule_id': 'RR-02-55aae13ba05649cb-16-F',
     'rule_mnxr': 'MNXR121985',
     'rule_mnxm': 'MNXM2781',
     'rule_score': 0.0,
     'right': {'TARGET_0000000001': 1},
     'left': {'CMPD_0000000003': 1, 'MNXM1': 1, 'MNXM738702': 1},
     'path_id': 1,
     'step': 1,
     'transformation_id': 'TRS_0_0_0'}}}},
 2: {'RR-02-7fd6432cb0671440-16-F': {'MNXR123092': {'MNXM1887': {'rule_id': 'RR-02-7fd6432cb0671440-16-F',
     'rule_mnxr': 'MNXR123092',
     'rule_mnxm': 'MNXM1887',
     'rule_score': 0.0,
     'right': {'CMPD_0000000003': 1},
     'left': {'MNXM13': 1, 'MNXM1401': 1},
     'path_id': 1,
     'step': 2,
     'transformation_id': 'TRS_0_1_0'}}}}}

In [52]:
input_subpath = parser.all_paths[1][0][1]
rp_path = parser.rp_paths[1][1]
match_threshold: float = 0.8

In [53]:
input_subpath

{'rule': 'RR-02-55aae13ba05649cb-16-F',
 'reaction': 'MNXR121985',
 'substrate': 'MNXM2781'}

In [62]:
rp_path

{'RR-02-55aae13ba05649cb-16-F': {'MNXR121985': {'MNXM2781': {'rule_id': 'RR-02-55aae13ba05649cb-16-F',
    'rule_mnxr': 'MNXR121985',
    'rule_mnxm': 'MNXM2781',
    'rule_score': 0.0,
    'right': {'TARGET_0000000001': 1},
    'left': {'CMPD_0000000003': 1, 'MNXM1': 1, 'MNXM738702': 1},
    'path_id': 1,
    'step': 1,
    'transformation_id': 'TRS_0_0_0'}}}}

In [6]:
data.rr_recipes['MNXR121985']

{'Equation': '1 MNXM1@MNXD1 + 1 MNXM1887@MNXD1 + 1 MNXM6@MNXD1 = 1 MNXM2781@MNXD1 + 1 MNXM5@MNXD1',
 'Description': '1 `H(+)` + 1 `3,4-dihydroxyphenylpyruvate` + 1 `NADPH` = 1 `(2R)-3-(3,4-dihydroxyphenyl)lactate` + 1 `NADP(+)`',
 'Direction': 1,
 'EC_number': ['1.1.1.237'],
 'Name': nan,
 'Type': nan,
 'UniProt_IDs': nan,
 'Additional_RIDs': 'metacyc:RXN-7632',
 'Main_left': 'MNXM1887',
 'Main_right': 'MNXM2781',
 'Secondary_left': 'MNXM1,MNXM6',
 'Secondary_right': 'MNXM5',
 'inchikey2_equation': '1 GPRLSGONYQIRFK-UHFFFAOYSA + 1 LQQFFJFGLSKYIR-UHFFFAOYSA + 1 ACFIXJIJDZMPPO-NNYOXOHSSA = 1 PAFLSMZLRSPALU-MRVPVSSYSA + 1 XJLXINKUBYWONI-NNYOXOHSSA',
 'inchikey_equation': '1 GPRLSGONYQIRFK-UHFFFAOYSA-N + 1 LQQFFJFGLSKYIR-UHFFFAOYSA-M + 1 ACFIXJIJDZMPPO-NNYOXOHSSA-J = 1 PAFLSMZLRSPALU-MRVPVSSYSA-M + 1 XJLXINKUBYWONI-NNYOXOHSSA-K',
 'main_reactants': {'MNXM1887': 1},
 'secondary_reactants': {'MNXM1': 1, 'MNXM738702': 1},
 'main_products': {'MNXM2781': 1},
 'secondary_products': {'MNXM5': 1}}

In [63]:
score

0.71875

In [60]:
import copy
import logging

from typing import Dict, Tuple, Any, Optional, Iterable, Literal, Set, Union, List
from typing import Callable, Dict, Any, Mapping, Optional

import pandas as pd
import numpy as np

In [61]:
subpath = copy.deepcopy(input_subpath)
rp_rule = subpath["rule"]
rp_rule_reac = subpath["reaction"]
rp_rule_substrate = subpath["substrate"]

# 1) Recover original recipe reactants/products (prefer rr_recipes, fall back to cache)
try:
    ori_reactants = parser.rr_recipes[rp_rule_reac]["main_reactants"] | parser.rr_recipes[rp_rule_reac]["secondary_reactants"]
    ori_products  = parser.rr_recipes[rp_rule_reac]["main_products"]  | parser.rr_recipes[rp_rule_reac]["secondary_products"]
except (KeyError, TypeError):
    try:
        base_id = parser.single_depr_mnxr(rp_rule_reac)
        ori_reactants = (
            parser.mnxr_prop[base_id]["main_reactants"]
            | parser.mnxr_prop[rp_rule_reac]["secondary_reactants"]
        )
        ori_products = (
            parser.mnxr_prop[base_id]["main_products"]
            | parser.mnxr_prop[rp_rule_reac]["secondary_products"]
        )
    except (KeyError, TypeError):
        raise KeyError(f"Cannot find original reactants/products for {rp_rule_reac}")

logging.debug(f"\t\tori_reactants: {ori_reactants}")
logging.debug(f"\t\tori_products: {ori_products}")

# 2) Build InChI pool for all species mentioned in the original recipe
ori_strc_inchi: Dict[str, str] = {}
left_out: list[str] = []
for mid in (ori_reactants | ori_products):
    inchi = parser.rp_strc.get(mid, {}).get("InChI")
    if not inchi:
        try:
            tmp = parser.mnxm_prop[mid]["InChI"]
            if not pd.isna(tmp):
                inchi = tmp
        except KeyError:
            inchi = None
    if inchi:
        ori_strc_inchi[mid] = inchi
    else:
        logging.warning(f"Cannot recover the InChI for {mid}")
        left_out.append(mid)

# 3) Identify the predicted major product on the RP2 right-hand side
rp_right = rp_path[rp_rule][rp_rule_reac][rp_rule_substrate]["right"]
if len(rp_right) != 1:
    raise KeyError(f"Multiple elements on RP2 right-hand side: {rp_right}")

rp_predict_strc_id = next(iter(rp_right.keys()))
target_best_mnxm, score = parser._best_inchi_match(parser.rp_strc[rp_predict_strc_id]["InChI"], ori_strc_inchi)

if score <= match_threshold and left_out:
    if len(left_out) == 1:
        logging.warning(f"Using unknown as main RHS metabolite: {left_out[0]}")
        target_best_mnxm = left_out[0]
    else:
        raise KeyError("Cannot confidently identify the RHS metabolite")

logging.debug(f"\t\t{rp_predict_strc_id} -> {target_best_mnxm}")

# 4) Determine orientation by checking where the target metabolite belongs
if target_best_mnxm in ori_products:
    reactant_dir, product_dir = "left", "right"
elif target_best_mnxm in ori_reactants:
    reactant_dir, product_dir = "right", "left"
else:
    raise KeyError("Cannot recognize reactant/product orientation")

# 5) Fetch predicted sides for this step
rp_reactants = rp_path[rp_rule][rp_rule_reac][rp_rule_substrate][reactant_dir]
rp_products  = rp_path[rp_rule][rp_rule_reac][rp_rule_substrate][product_dir]
logging.debug(f"\t\trp_reactants: {rp_reactants}")
logging.debug(f"\t\trp_products: {rp_products}")

# 6) Map predicted species to original IDs using InChI similarity (reactants)
reactants_rp2ori: Dict[str, str] = {}
for mid in rp_reactants:
    if mid not in ori_reactants:
        inchi = parser.rp_strc.get(mid, {}).get("InChI")
        if inchi:
            best_mnxm, _ = parser._best_inchi_match(inchi, ori_strc_inchi)
            reactants_rp2ori[mid] = best_mnxm

# 7) Map predicted species to original IDs (products)
products_rp2ori: Dict[str, str] = {rp_predict_strc_id: target_best_mnxm}
for mid in rp_products:
    if mid not in ori_products and mid != rp_predict_strc_id:
        inchi = parser.rp_strc.get(mid, {}).get("InChI")
        if inchi:
            best_mnxm, _ = parser._best_inchi_match(inchi, ori_strc_inchi)
            products_rp2ori[mid] = best_mnxm

logging.debug(f"\t\tConverted rp_reactants: {[reactants_rp2ori.get(i, i) for i in rp_reactants.keys()]}")
logging.debug(f"\t\tConverted rp_products: {[products_rp2ori.get(i, i) for i in rp_products.keys()]}")

# 8) Determine which species are missing (to add from the recipe)
to_add_reactants = set(ori_reactants.keys()) - {reactants_rp2ori.get(i, i) for i in rp_reactants.keys()}
to_add_products  = set(ori_products.keys())  - {products_rp2ori.get(i, i)  for i in rp_products.keys()}
logging.debug(f"\t\tto_add_reactants: {to_add_reactants}")
logging.debug(f"\t\tto_add_products: {to_add_products}")

# 9) Populate the completed subpath
subpath["reactants"] = copy.deepcopy(rp_reactants)
subpath["products"]  = copy.deepcopy(rp_products)
subpath["transformation_id"] = rp_path[rp_rule][rp_rule_reac][rp_rule_substrate]["transformation_id"]

# Add any missing recipe species with their stoichiometries
for mid in to_add_products:
    subpath["products"][mid] = ori_products[mid]
for mid in to_add_reactants:
    subpath["reactants"][mid] = ori_reactants[mid]

logging.debug(f"\t\tfull reactants: {subpath['reactants']}")
logging.debug(f"\t\tfull products: {subpath['products']}")
logging.debug("\t -------")

KeyError: 'Cannot confidently identify the RHS metabolite'

In [64]:
ori_reactants

{'MNXM2781': 1, 'MNXM137': 1}

In [65]:
ori_products

{'MNXM1887': 1, 'MNXM162231': 1}

In [66]:
ori_strc_inchi

{'MNXM2781': 'InChI=1S/C9H10O5/c10-6-2-1-5(3-7(6)11)4-8(12)9(13)14/h1-3,8,10-12H,4H2,(H,13,14)/p-1/t8-/m1/s1',
 'MNXM1887': 'InChI=1S/C9H8O5/c10-6-2-1-5(3-7(6)11)4-8(12)9(13)14/h1-3,10-11H,4H2,(H,13,14)/p-1'}

In [67]:
left_out

['MNXM137', 'MNXM162231']

In [68]:
rp_right

{'TARGET_0000000001': 1}

In [69]:
rp_predict_strc_id

'TARGET_0000000001'

In [70]:
target_best_mnxm

'MNXM2781'

In [ ]:
from metaxime.utils import merge_models
merged_model = merge_models(
    model,
    cobrak_biopath_iml1515.model,
)

In [2]:
data = Data()

In [6]:
import pubchempy as pcp


In [133]:
a = pcp.get_compounds(identifier='FCXZBWSIAGGPCB-UHFFFAOYSA-N', namespace='inchikey', as_dataframe=False)

In [137]:
a = a[0].to_dict()

In [139]:
a['cid']

528

In [131]:
import requests
cid = 528
r = requests.post(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/xrefs/SBURL/JSON')
res_list = r.json()

xref = {}
xref['pubchem'] = [str(cid)]
for url in res_list['InformationList']['Information'][0]['SBURL']:
    if 'https://biocyc.org/compound?orgid=META&id=' in url:
        if 'biocyc' not in xref:
            xref['biocyc'] = []
        xref['biocyc'].append(url.replace('https://biocyc.org/compound?orgid=META&id=', ''))
    if 'http://www.hmdb.ca/cidbolites/' in url:
        if 'hmdb' not in xref:
            xref['hmdb'] = []
        xref['hmdb'].append(url.replace('http://www.hmdb.ca/cidbolites/', ''))
    if 'http://www.genome.jp/dbget-bin/www_bget?cpd:' in url:
        if 'kegg.compound' not in xref:
            xref['kegg.compound'] = []
        xref['kegg.compound'].append(url.replace('http://www.genome.jp/dbget-bin/www_bget?cpd:', ''))
    if 'http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:' in url:
        if 'chebi' not in xref:
            xref['chebi'] = []
        xref['chebi'].append('CHEBI:'+url.replace('http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:', ''))
        xref['chebi'].append(url.replace('http://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:', ''))

In [132]:
xref

{'pubchem': ['528'], 'chebi': ['CHEBI:7671', '7671']}

In [128]:
data.chebim_mnxm['7671']

np.str_('MNXM731456')

In [130]:
data.mnxm_xref('MNXM1371311')

({'name': 'O-acetyl-D-homoserine',
  'reference': 'chebi:37034',
  'formula': 'C6H11NO4',
  'charge': 0.0,
  'mass': 161.157,
  'InChI': 'InChI=1S/C6H11NO4/c1-4(8)11-3-2-5(7)6(9)10/h5H,2-3,7H2,1H3,(H,9,10)/t5-/m1/s1',
  'InChIKey': 'FCXZBWSIAGGPCB-RXMQYKEDSA-N',
  'SMILES': 'CC(=O)OCC[C@@H](N)C(=O)O',
  'xref': {'metanetx.chemical': ['MNXM1371311']}},
 'MNXM1371311')

In [99]:
pcp.get(
    identifier=528,
    namespace='cid',
    domain='compound',
    searchtype='xrefs',
)

BadRequestError: 'PUGREST.BadRequest'

In [96]:
pcp.request(
    identifier=528,
    namespace='cid',
    domain='compound',
    searchtype='xref',
)

BadRequestError: 'PUGREST.BadRequest'

In [93]:
c = pcp.Compound.from_cid(528, searchtype="xref")

BadRequestError: 'PUGREST.BadRequest'

In [90]:
c.

['4-acetyloxy-2-aminobutanoic acid',
 'RefChem:1076025',
 '4-acetyloxy-2-azaniumylbutanoate',
 'Acetylhomoserine',
 '4-Acetoxy-2-aminobutanoic acid',
 'Homoserine acetate',
 'O-acetyl-dl-homoserine',
 'SCHEMBL302495',
 '4-acetoxy-2-amino-butanoic acid',
 'HAA54067',
 '4-(acetyloxy)-2-aminobutanoic acid',
 'Q27107556']

In [84]:
c = a[0]

In [85]:
c.record.keys()

dict_keys(['id', 'atoms', 'bonds', 'stereo', 'coords', 'charge', 'props', 'count'])

In [86]:
len(c.record['props'])

22

In [88]:
c.

Compound(528)

In [87]:
c.record['props'][21]

{'urn': {'label': 'Weight',
  'name': 'MonoIsotopic',
  'datatype': 1,
  'version': '2.1',
  'software': 'PubChem',
  'source': 'ncbi.nlm.nih.gov',
  'release': '2021.05.07'},
 'value': {'sval': '161.06880783'}}

In [47]:
c = c.to_dict()

In [48]:
c['records']

KeyError: 'records'

In [46]:
c.to_dict()

{'atom_stereo_count': 1,
 'atoms': [{'aid': 1, 'number': 8, 'element': 'O', 'x': 3.732, 'y': -1.25},
  {'aid': 2, 'number': 8, 'element': 'O', 'x': 6.3301, 'y': 1.25},
  {'aid': 3, 'number': 8, 'element': 'O', 'x': 5.4641, 'y': 2.75},
  {'aid': 4, 'number': 8, 'element': 'O', 'x': 2, 'y': -1.25},
  {'aid': 5, 'number': 7, 'element': 'N', 'x': 3.732, 'y': 1.75},
  {'aid': 6, 'number': 6, 'element': 'C', 'x': 4.5981, 'y': 0.25},
  {'aid': 7, 'number': 6, 'element': 'C', 'x': 4.5981, 'y': 1.25},
  {'aid': 8, 'number': 6, 'element': 'C', 'x': 3.732, 'y': -0.25},
  {'aid': 9, 'number': 6, 'element': 'C', 'x': 5.4641, 'y': 1.75},
  {'aid': 10, 'number': 6, 'element': 'C', 'x': 2.866, 'y': -1.75},
  {'aid': 11, 'number': 6, 'element': 'C', 'x': 2.866, 'y': -2.75},
  {'aid': 12, 'number': 1, 'element': 'H', 'x': 4.8101, 'y': -0.3326},
  {'aid': 13, 'number': 1, 'element': 'H', 'x': 5.2087, 'y': 0.3577},
  {'aid': 14, 'number': 1, 'element': 'H', 'x': 4.5981, 'y': 1.87},
  {'aid': 15, 'number':

In [38]:
c['record'].keys()

dict_keys(['id', 'atoms', 'bonds', 'stereo', 'coords', 'charge', 'props', 'count'])

In [39]:
c['tpsa']

89.6

In [2]:
bio_model = ModelBuilder('iML1515.xml')


KeyboardInterrupt



In [9]:
data.exact_pubchem_search(query='FCXZBWSIAGGPCB-UHFFFAOYSA-N', itype='inchikey')

KeyError: 'Multiple cids [Compound(528), Compound(74413607)] for FCXZBWSIAGGPCB-UHFFFAOYSA-N'

In [2]:
from metaxime.parser import ParserRP2
parser = ParserRP2(
    rp2_scope_path='bug/out_scope.csv',
    rp2_cmp_path='bug/out_compounds.csv',
    rp2_paths_path='bug/out_paths.csv',
)
all_models = parser.return_rp2_models(compartment_id='c')

In [32]:
for path_id in all_models:
    for sub_path_id in all_models[path_id]:
        for m in all_models[path_id][sub_path_id].metabolites:
            try:
                int(m.charge)
            except ValueError:
                m.charge = 0.0
for path_id in all_models:
    for sub_path_id in all_models[path_id]:
        for m in all_models[path_id][sub_path_id].metabolites:
            if not isinstance(m.formula, str) or m.formula in ('nan', 'None', 'NaN', ''):
                m.formula = None

In [51]:
from cobra.io import write_sbml_model

for path_id in all_models:
    for sub_path_id in all_models[path_id]:
        print(f'-> {path_id} -- {sub_path_id}')
        print(all_models[path_id][sub_path_id].id)
        write_sbml_model(all_models[path_id][sub_path_id], f"test_out/{all_models[path_id][sub_path_id].id}.xml")

-> 1 -- 0
rp2_1_0
-> 1 -- 1
rp2_1_1
-> 1 -- 2
rp2_1_2
-> 2 -- 0
rp2_2_0
-> 3 -- 0
rp2_3_0
-> 4 -- 0
rp2_4_0
-> 5 -- 0
rp2_5_0
-> 5 -- 1
rp2_5_1
-> 6 -- 0
rp2_6_0
-> 6 -- 1
rp2_6_1
-> 7 -- 0
rp2_7_0
-> 7 -- 1
rp2_7_1
-> 7 -- 2
rp2_7_2
-> 7 -- 3
rp2_7_3
-> 7 -- 4
rp2_7_4
-> 7 -- 5
rp2_7_5
-> 7 -- 6
rp2_7_6
-> 7 -- 7
rp2_7_7
-> 8 -- 0
rp2_8_0
-> 8 -- 1
rp2_8_1
-> 8 -- 2
rp2_8_2
-> 8 -- 3
rp2_8_3
-> 9 -- 0
rp2_9_0
-> 9 -- 1
rp2_9_1
-> 9 -- 2
rp2_9_2
-> 9 -- 3
rp2_9_3
-> 9 -- 4
rp2_9_4
-> 9 -- 5
rp2_9_5
-> 9 -- 6
rp2_9_6
-> 9 -- 7
rp2_9_7
-> 9 -- 8
rp2_9_8
-> 9 -- 9
rp2_9_9
-> 9 -- 10
rp2_9_10
-> 9 -- 11
rp2_9_11
-> 9 -- 12
rp2_9_12
-> 9 -- 13
rp2_9_13
-> 9 -- 14
rp2_9_14
-> 9 -- 15
rp2_9_15
-> 10 -- 0
rp2_10_0
-> 10 -- 1
rp2_10_1
-> 11 -- 0
rp2_11_0
-> 11 -- 1
rp2_11_1
-> 11 -- 2
rp2_11_2
-> 11 -- 3
rp2_11_3
-> 11 -- 4
rp2_11_4
-> 11 -- 5
rp2_11_5
-> 11 -- 6
rp2_11_6
-> 11 -- 7
rp2_11_7
-> 12 -- 0
rp2_12_0
-> 12 -- 1
rp2_12_1
-> 12 -- 2
rp2_12_2
-> 12 -- 3
rp2_12_3
-> 14 -- 0
rp2_14_0
-> 1

In [63]:
from cobra.io import read_sbml_model, write_sbml_model

model = all_models[7][3]
cobrak_biopath_iml1515 = read_sbml_model('cobrak_iML1515.xml')

13-10-2025 11:28:08.038 ERROR sbml - _sbml_to_model: '' is not a valid SBML 'SId'.
13-10-2025 11:28:09.854 ERROR sbml - _sbml_to_model: No objective coefficients in model. Unclear what should be optimized


In [64]:
from metaxime.utils import merge_models
merged_model = merge_models(
    model,
    cobrak_biopath_iml1515,
)

In [73]:
write_sbml_model(model, 'rp2_model.xml')

In [67]:
a = [i for i in merged_model.reactions if 'TRS' in i.id]

In [72]:
a

[<Reaction GLUTRS_ENZ_b2400 at 0x14eb169e0>,
 <Reaction TRSARr_ENZ_b3125_REV at 0x330e863e0>,
 <Reaction TRSARr_ENZ_b3125_FWD at 0x330e864a0>,
 <Reaction TRS_0_0_5 at 0x14f3ff820>,
 <Reaction TRS_0_1_0 at 0x148958b20>,
 <Reaction TRS_0_2_30 at 0x148958df0>]

In [47]:
model.metabolites.get_by_id('MNXM8975_c').formula

nan

In [43]:
for m in model.metabolites:
    print(m.id)
    print(m.charge)
    print(m.formula)
    print('------')

TARGET_0000000001_c
0.0
None
------
MNXM732923_c
0.0
C3H7NO2Se
------
CMPD_0000000011_c
0.0
None
------
MNXM26_c
-1.0
C2H3O2
------
WATER_c
0.0
H2O
------
CMPD_0000000014_c
0.0
None
------
MNXM159_c
-1.0
C4H5O3
------
MNXM729302_c
1.0
H4N
------
MNXM1105731_c
0.0
C3H7NO2
------
MNXM741485_c
-2.0
Se
------
MNXM1368697_c
-1.0
HSe
------
MNXM8975_c
0.0
nan
------
MNXM1102419_c
0.0
nan
------


In [20]:
write_sbml_model(m, 'test_out/tmp.xml')

TypeError: Wrong number or type of arguments for overloaded function 'FbcSpeciesPlugin_setCharge'.
  Possible C/C++ prototypes are:
    FbcSpeciesPlugin::setCharge(int)
    FbcSpeciesPlugin::setCharge(double)


In [11]:
parser.return_rp2_models?

Signature:
parser.return_rp2_models(
    compartment_id: 'str' = 'c',
    reaction_lower_bound: 'float' = 0.0,
    reaction_upper_bound: 'float' = 1000.0,
) -> 'Dict[int, Dict[int, Model]]'
Docstring:
Build one COBRA model per (path_num, subpath) from RetroPath-like data.

Args:
    compartment_id: COBRA compartment ID to assign to created metabolites.
    reaction_lower_bound: Lower bound applied to every reaction.
    reaction_upper_bound: Upper bound applied to every reaction.

Returns:
    Dict mapping path_num -> {subpath_index -> cobra.Model}.
File:      ~/workspace/metaxime/metaxime/parser.py
Type:      method

In [4]:
all_models = parser.return_rp2_models()

In [5]:
model = all_models[6][0]

In [6]:
biopath_iml1515 = ModelBuilder(path_to_model='iML1515.xml', use_progressbar=True)

Updating the reaction annotations: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2712/2712 [00:41<00:00, 65.53it/s]


In [14]:
cobrak_biopath_iml1515 = ModelBuilder(path_to_model='cobrak_iML1515.xml', use_progressbar=True)

09-10-2025 09:24:21.800 ERROR sbml - _sbml_to_model: '' is not a valid SBML 'SId'.
09-10-2025 09:24:23.167 ERROR sbml - _sbml_to_model: No objective coefficients in model. Unclear what should be optimized
Updating the reaction annotations: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3388/3388 [00:51<00:00, 65.25it/s]


In [17]:
cobrak_biopath_iml1515.model.reactions.NDPK5_ENZ_b2518_REV.annotation

{'sbo': 'SBO:0000176',
 'bigg.reaction': 'NDPK5',
 'biocyc': 'META:DGDPKIN-RXN',
 'ec-code': '2.7.4.6',
 'kegg.reaction': 'R01857',
 'metanetx.reaction': 'MNXR96118',
 'rhea': ['27692', '27690', '27691', '27693'],
 'sabiork': '183',
 'seed.reaction': 'rxn01353',
 'cobrak_id_V0': 'NDPK5_ENZ_b2518_REV',
 'cobrak_dG0_V0': '-1.85251131821542',
 'cobrak_k_cat_V0': '1e+20',
 'cobrak_k_cat_references_V0': '[]',
 'cobrak_k_ms_V0': '{}',
 'cobrak_k_m_references_V0': '{}',
 'cobrak_k_is_V0': '{}',
 'cobrak_k_i_references_V0': '{}',
 'cobrak_k_as_V0': '{}',
 'cobrak_k_a_references_V0': '{}',
 'cobrak_special_stoichiometries_V0': '{}',
 'mnx_equation': '1 MNXM344@MNXD1 + 1 MNXM40333@MNXD1 = 1 MNXM3@MNXD1 + 1 MNXM436@MNXD1',
 'reference': 'rh:27690',
 'is_balanced': 'B',
 'inchikey2_equation': '1 HAAZLUGHYHWQIW-KVQBGUIXSA + 1 XTWYTFMLZFPYCI-KQYNXXCUSA = 1 ZKHQWZAMYRWXGA-KQYNXXCUSA + 1 CIKGWCTVFSRMJU-KVQBGUIXSA',
 'inchikey_equation': '1 HAAZLUGHYHWQIW-KVQBGUIXSA-J + 1 XTWYTFMLZFPYCI-KQYNXXCUSA-K = 

In [18]:
import logging
from cobra import Model, Reaction, Metabolite
from typing import Dict

def merge_models(
    source_model: Model,
    input_target_model: Model,
) -> Model:
    """Merge a COBRApy model into another by matching metabolites via annotation overlap.

    For each metabolite in the source model, this function tries to find a match in the
    target model based on overlapping annotations. If a match is found, the metabolite
    in the source model is mapped to the corresponding target metabolite. Reactions
    from the source model are then copied and added to the target model using this mapping.

    Args:
        source_model (Model): The model whose reactions should be merged into the target.
        target_model (Model): The model to which matching reactions are added.

    Returns:
        Dict[str, str]: Mapping of source metabolite IDs to target metabolite IDs.
    """
    def _annotations_overlap(a: Dict[str, Any], b: Dict[str, Any]) -> bool:
        """
        Return True if there is at least one overlapping value for any key present in both dicts.
        Values may be strings or iterables of strings.
        """
        def _to_norm_set(v: Any) -> Set[str]:
            """Normalize a value (str or iterable) to a lowercase, stripped set of strings."""
            if v is None:
                return set()
            if isinstance(v, (list, tuple, set)):
                items: Iterable[Any] = v
            else:
                items = [v]
            return {str(x).strip().lower() for x in items if x is not None}
        for key in set(a) & set(b):
            if _to_norm_set(a[key]) & _to_norm_set(b[key]):
                return True
        return False
    target_model = input_target_model.copy()
    gen_ori_convert_metabolites: Dict[str, str] = {}

    # Match metabolites based on annotation overlap
    for gen_m in source_model.metabolites:
        for ori_m in target_model.metabolites:
            if annotations_overlap(gen_m.annotation, ori_m.annotation):
                logging.debug(f"{gen_m.id} matches {ori_m.id}")
                gen_ori_convert_metabolites[gen_m.id] = ori_m.id
                break
    # Copy reactions with mapped metabolites
    new_reactions = []
    for r in source_model.reactions:
        reaction = Reaction(r.id)
        reaction.name = r.name
        reaction.lower_bound = r.lower_bound
        reaction.upper_bound = r.upper_bound
        reaction.annotation = r.annotation
        reac_meta_dict = {}
        for met, coeff in r.metabolites.items():
            mapped_id = gen_ori_convert_metabolites.get(met.id, met.id)
            try:
                meta = target_model.metabolites.get_by_id(mapped_id)
            except KeyError:
                meta = source_model.metabolites.get_by_id(met.id)
            reac_meta_dict[meta] = coeff
        reaction.add_metabolites(reac_meta_dict)
        new_reactions.append(reaction)
        
    target_model.add_reactions(new_reactions)
    logging.info(f"Added {len(new_reactions)} reactions to {target_model.id or 'target model'}")
    return target_model

In [19]:
merged_model = merge_models(
    model,
    cobrak_biopath_iml1515.model,
)

In [2]:
from metaxime.utils import merge_models
merged_model = merge_models(
    model,
    cobrak_biopath_iml1515.model,
)

In [4]:
merge_models

<function metaxime.utils.merge_models(source_model: cobra.core.model.Model, input_target_model: cobra.core.model.Model) -> cobra.core.model.Model>

In [12]:
m = all_models[10][0]

In [13]:
m.reactions.get_by_id("TRS_0_2_29").annotation

{'rp_score': np.float64(0.0),
 'rp_step': 3,
 'rp_id': 'RR-02-95f6aa357bb05915-16-F',
 'metanetx.reaction': 'MNXR152862',
 'ec-code': ['2.5.1.47']}

In [31]:
m.

{'TRS_0_0_0': {'score': np.float64(3.8457180179666586),
  'reaction_smiles': '[H]OC(=O)C([H])(N([H])[H])C([H])([H])C([H])([H])OC(=O)C([H])([H])[H]>>O=C=O.[H]OC(=O)C([H])([H])C([H])([H])C([H])([H])N([H])[H].[H]OP(=O)(O[H])OC([H])([H])C([H])([H])c1sc([H])nc1C([H])([H])[H]'},
 'TRS_0_0_1': {'score': np.float64(2.0170333392987803),
  'reaction_smiles': '[H]OC(=O)C([H])(N([H])[H])C([H])([H])C([H])([H])OC(=O)C([H])([H])[H]>>[H]OC(=NC([H])([H])C([H])([H])SC(=O)C([H])([H])[H])C([H])([H])C([H])([H])N=C(O[H])C([H])(O[H])C(C([H])([H])[H])(C([H])([H])[H])C([H])([H])OP(=O)(O[H])OP(=O)(O[H])OC([H])([H])C1([H])OC([H])(n2c([H])nc3c(N([H])[H])nc([H])nc32)C([H])(O[H])C1([H])OP(=O)(O[H])O[H].[H]OC(=O)C([H])(N([H])[H])C([H])([H])C([H])([H])O[H]'},
 'TRS_0_0_2': {'score': np.float64(0.0),
  'reaction_smiles': '[H]OC(=O)C([H])(N([H])[H])C([H])([H])C([H])([H])OC(=O)C([H])([H])[H]>>[H]OC(=O)C([H])(N([H])[H])C([H])([H])C([H])([H])SC([H])([H])[H].[H]OC(=O)C([H])([H])[H]'},
 'TRS_0_0_3': {'score': np.float64(0.6